<a href="https://colab.research.google.com/github/younesabdolmalaky/LTR-on-torob-data/blob/main/notebooks/FeatureExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import json
import pickle
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence

In [3]:
def read_json_lines(path, n_lines=None):
    """Creates a generator which reads and returns lines of
    a json lines file, one line at a time, each as a dictionary.
    
    This could be used as a memory-efficient alternative of `pandas.read_json`
    for reading a json lines file.
    """
    with open(path, 'r') as f:
        for i, line in enumerate(f):
            if n_lines == i:
                break
            yield json.loads(line)
            
aggregated_search_data_path = '/content/drive/MyDrive/torob/output_data/aggregated_search_data.jsonl'
preprocessed_products_path = '/content/drive/MyDrive/torob/output_data/preprocessed_products.jsonl'
preprocessed_test_queries_path = '/content/drive/MyDrive/torob/output_data/preprocessed_test_queries.jsonl'

In [5]:
products_data_df = pd.DataFrame(read_json_lines(preprocessed_products_path))

In [6]:
MAX_FEATURES = 12000
tokenizer = Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(products_data_df['title_normalized'])
doc = tokenizer.texts_to_sequences(products_data_df['title_normalized'])
del products_data_df

In [7]:
MAX_LENGTH = max(len(train_ex) for train_ex in doc)

In [8]:
doc = pad_sequences(doc, maxlen=MAX_LENGTH)
pickle.dump(doc, open("/content/drive/MyDrive/torob/Features/doc_sequences.h5", "wb"))
del doc

In [9]:
aggregated_searches_df = pd.DataFrame(read_json_lines(aggregated_search_data_path, n_lines=None))
train_query = tokenizer.texts_to_sequences(aggregated_searches_df['raw_query_normalized'])
train_query = pad_sequences(train_query, maxlen=MAX_LENGTH)
pickle.dump(train_query, open("/content/drive/MyDrive/torob/Features/train_query_sequences.h5", "wb"))
del train_query
del aggregated_searches_df 

In [11]:
test_offline_queries_df = pd.DataFrame(read_json_lines(preprocessed_test_queries_path))
test_query =  tokenizer.texts_to_sequences(test_offline_queries_df['raw_query_normalized'])
test_query = pad_sequences(test_query, maxlen=MAX_LENGTH)
pickle.dump(test_query, open("/content/drive/MyDrive/torob/Features/test_query_sequences.h5", "wb"))
del test_query
del test_offline_queries_df

In [12]:
%reset -f

In [1]:
import os
import json
import pickle
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
def read_json_lines(path, n_lines=None):
    """Creates a generator which reads and returns lines of
    a json lines file, one line at a time, each as a dictionary.
    
    This could be used as a memory-efficient alternative of `pandas.read_json`
    for reading a json lines file.
    """
    with open(path, 'r') as f:
        for i, line in enumerate(f):
            if n_lines == i:
                break
            yield json.loads(line)
            
aggregated_search_data_path = '/content/drive/MyDrive/torob/output_data/aggregated_search_data.jsonl'
preprocessed_products_path = '/content/drive/MyDrive/torob/output_data/preprocessed_products.jsonl'
preprocessed_test_queries_path = '/content/drive/MyDrive/torob/output_data/preprocessed_test_queries.jsonl'

In [5]:
VOCAB_SIZE = 8196
EMBEDDING_DIM = 256
NUM_TRAIN_SAMPLES = None

In [6]:
vectorizer = TfidfVectorizer(max_features=VOCAB_SIZE, lowercase=True, use_idf=True)
products_data_df = pd.DataFrame(read_json_lines(preprocessed_products_path))
products_tfidf = vectorizer.fit_transform(products_data_df['title_normalized'])
del products_data_df
pickle.dump(products_tfidf, open("/content/drive/MyDrive/torob/Features/doc_tfidf.h5", "wb"))
del products_tfidf

In [7]:
aggregated_searches_df = pd.DataFrame(read_json_lines(aggregated_search_data_path, n_lines=None))
queries_train_tfidf = vectorizer.transform(aggregated_searches_df['raw_query_normalized'])
pickle.dump(queries_train_tfidf, open("/content/drive/MyDrive/torob/Features/train_query_tfidf.h5", "wb"))
del queries_train_tfidf
del aggregated_searches_df

In [8]:
test_offline_queries_df = pd.DataFrame(read_json_lines(preprocessed_test_queries_path))
queries_test_tfidf = vectorizer.transform(test_offline_queries_df['raw_query_normalized'])
pickle.dump(queries_test_tfidf, open("/content/drive/MyDrive/torob/Features/test_query_tfidf.h5", "wb"))
del queries_test_tfidf
del test_offline_queries_df